In [1]:
import numpy as np
from scipy.optimize import root
from vpython import *
import math
import random
import time

<IPython.core.display.Javascript object>

[RSPR.PNG](attachment:RSPR.PNG)

In [2]:
# Function of quaternion
def QuaterTimes(q1, q2):
    a, b, c, d = q1;
    e, f, g, h = q2;
    w = a*e - b*f - c*g - d*h;
    x = b*e + a*f - d*g + c*h;
    y = c*e + d*f + a*g - b*h;
    z = d*e - c*f + b*g + a*h;
    
    return np.array([w, x, y, z])

def QuaterConj(q):
    w, x, y, z = q;
    return np.array([w, -x, -y, -z])

In [3]:
def distance(a, b):
    
    dis = math.sqrt((a[0] - b[0])*(a[0] - b[0]) + (a[1] - b[1])*(a[1] - b[1]) + (a[2] - b[2])*(a[2] - b[2]));
    
    return dis


In [4]:
# This function is for randomly gain a new vector candidate that is vertical to the input vec
#为什么我会用random.uniform啊， 我他妈看不懂了=。=

def RandomChooseVerticalVector (vec):
    candidate = np.array([0,0,0],dtype=np.float64);
    if vec[2] == 0:
        if vec[0]*vec[1] != 0:
            candidate[0] = random.uniform(-6.0, 6.0);
            candidate[1] = -vec[0]*candidate[0]/u24[1];
            candidate[2] = random.uniform(-6.0, 6.0);
        elif vec[0] == 0:
            candidate[0] = random.uniform(-6.0, 6.0);
            candidate[1] = 0;
            candidate[2] = random.uniform(-6.0, 6.0);  
        else:
            candidate[0] = 0;
            candidate[1] = random.uniform(-6.0, 6.0);
            candidate[2] = random.uniform(-6.0, 6.0);
    else:
        candidate[0] = random.uniform(-6.0, 6.0);
        candidate[1] = random.uniform(-6.0, 6.0);
        candidate[2] = (-candidate[0]*vec[0]-candidate[1]*vec[1])/vec[2];

                        
    return candidate

In [5]:
def RigidbodyMesh (upara, new_pos_3):
    
    mesh_data = np.zeros((27,3), dtype=np.float64);
    
    center = 0.5*(upara[0:3]-new_pos_3) + new_pos_3;
    #x axis
    x_rigid = np.cross((new_pos_3 - upara[0:3]),upara[6:9]);
    x_rigid = x_rigid/np.linalg.norm(x_rigid);
    #y axis
    y_rigid = new_pos_3 - upara[0:3];
    y_rigid = y_rigid/np.linalg.norm(y_rigid);
    #z axis
    z_rigid = np.cross(x_rigid,y_rigid);
    z_rigid = z_rigid/np.linalg.norm(z_rigid);

    #mesh, unit = 2
    mesh_data[0,:] = center; #[0,0,0]

    mesh_data[1,:] = center + 2*z_rigid; #[0,0,1]
    mesh_data[2,:] = center - 2*z_rigid; #[0,0,-1]
    
    mesh_data[3,:] = center + 2*x_rigid; #[1,0,0]
    mesh_data[4,:] = center - 2*x_rigid; #[-1,0,0]
    
    mesh_data[5,:] = center + 2*y_rigid; #[0,1,0]
    mesh_data[6,:] = center - 2*y_rigid; #[0,-1,0]
    
    
    mesh_data[7,:] = center + 2*x_rigid + 2*y_rigid; #[1,1,0]
    mesh_data[8,:] = center + 2*x_rigid - 2*y_rigid; #[1,-1,0]
    mesh_data[9,:] = center - 2*x_rigid + 2*y_rigid; #[-1,1,0]
    mesh_data[10,:] = center - 2*x_rigid - 2*y_rigid; #[-1,-1,0]
    
    mesh_data[11,:] = center + 2*x_rigid + 2*z_rigid; #[1,0,1]
    mesh_data[12,:] = center + 2*x_rigid - 2*z_rigid; #[1,0,-1]
    mesh_data[13,:] = center - 2*x_rigid + 2*z_rigid; #[-1,0,1]
    mesh_data[14,:] = center - 2*x_rigid - 2*z_rigid; #[-1,0,-1]
    
    mesh_data[15,:] = center + 2*y_rigid + 2*z_rigid; #[0,1,1]
    mesh_data[16,:] = center + 2*y_rigid - 2*z_rigid; #[0,1,-1]
    mesh_data[17,:] = center - 2*y_rigid + 2*z_rigid; #[0,-1,1]
    mesh_data[18,:] = center - 2*y_rigid - 2*z_rigid; #[0,-1,-1]
    
    mesh_data[19,:] = center + 2*x_rigid + 2*y_rigid + 2*z_rigid; #[1,1,1]
    mesh_data[20,:] = center + 2*x_rigid + 2*y_rigid - 2*z_rigid; #[1,1,-1]
    mesh_data[21,:] = center + 2*x_rigid - 2*y_rigid + 2*z_rigid; #[1,-1,1]
    mesh_data[22,:] = center - 2*x_rigid + 2*y_rigid + 2*z_rigid; #[-1,1,1]
    mesh_data[23,:] = center + 2*x_rigid - 2*y_rigid - 2*z_rigid; #[1,-1,-1]
    mesh_data[24,:] = center - 2*x_rigid - 2*y_rigid + 2*z_rigid; #[-1,-1,1]
    mesh_data[25,:] = center - 2*x_rigid + 2*y_rigid - 2*z_rigid; #[-1,1,-1]
    mesh_data[26,:] = center - 2*x_rigid - 2*y_rigid - 2*z_rigid; #[-1,-1,-1]
    
    return mesh_data
    

In [6]:
def ComputeConfig (fpara, pos3, pos4, u5):
    
    #configuration of the mechanism, the order is l13, l34, l25, 
    #ang_25_u2, ang_25_u5
    #ang_u2_u5, ang_43_u5
    config = np.array([0,0,0,0,0,0,0], dtype=np.float64);
    
    config[0] = np.linalg.norm(fpara[1,:]-pos3); #l13
    config[1] = np.linalg.norm(pos4-pos3); #l34
    config[2] = np.linalg.norm(fpara[2,:]-pos4); #l25
    
    
    #ang_u2_u25
    u2_u25 = np.dot(fpara[4,:], -fpara[2,:]+pos4)/np.linalg.norm(-fpara[2,:]+pos4);
    u2_u25 = (u2_u25 if u2_u25 > -np.pi else u2_u25+np.pi) if u2_u25 < np.pi else u2_u25-np.pi;#make sure the ang is in ((0,pi)
    config[3] = np.arccos(u2_u25);
    
    #ang_u5_u52
    u5_u52 = np.dot(Initial_u5, fpara[2,:]-pos4)/np.linalg.norm(fpara[2,:]-pos4);
    u5_u52 = (u5_u52 if u5_u52 > -np.pi else u5_u52+np.pi) if u5_u52 < np.pi else u5_u52-np.pi;#make sure the ang is in ((0,pi)
    config[4] = np.arccos(u5_u52);
    
    
    #ang_u2_u5
    u2_u5 = np.dot(fpara[4,:], u5)/np.linalg.norm(fpara[4,:]);
    u2_u5 = (u2_u5 if u2_u5 > -np.pi else u2_u5+np.pi) if u2_u5 < np.pi else u2_u5-np.pi;#make sure the ang is in ((0,pi)
    config[5] = np.arccos(u2_u5);

    #ang_u34_u5
    u34_u5 = np.dot(pos3-pos4, u5)/np.linalg.norm(pos3-pos4);
    u34_u5 = (u34_u5 if u34_u5 > -np.pi else u34_u5+np.pi) if u34_u5 < np.pi else u34_u5-np.pi;#make sure the ang is in ((0,pi)
    config[6] = np.arccos(u34_u5);
    
    return config

In [11]:
def solve_equations(fpara, config, guess, new):
    
    def ConstrainFun (upara):
        upara = np.reshape(np.array(upara),(3,3));
        f1 = distance(new, upara[0,:]) - config[1]; #l_34
        f2 = distance(fpara[2,:], upara[1,:]) - config[2]; #l_25
        f3 = np.dot((new-upara[0,:]),upara[2,:]) - config[1]*np.cos(config[6]); #ang_43_u5
        f4 = np.dot(upara[2,:], fpara[4,:]) - np.cos(config[5]); #ang_u2_u5
        f5 = np.dot(upara[2,:], (fpara[2,:]-upara[1,:])) - config[2]*np.cos(config[4]); #ang_u5_u52
        f6 = np.dot(fpara[4,:], (upara[1,:]-fpara[2,:])) - config[2]*np.cos(config[3]); #ang_u2_u25
        f7 = math.sqrt((upara[2,0] - 0)*(upara[2,0] - 0) + (upara[2,1] - 0)*(upara[2,1] - 0) + (upara[2,2] - 0)*(upara[2,2] - 0)) - 1; #u5 is a unit vector
        #make sure u_45 and u_5 are parallel, f7-f9_2 has 3 valid constraint
        u45 = upara[1,:]-upara[0,:];
        cross_45_u5 = np.cross((upara[1,:]-upara[0,:]),upara[2,:]); #the cross product of u_45 and u_5
        if abs(upara[2,2]) <= 10e-8:
            f8 = u45[2];
            f9 = cross_45_u5[2];
            #print("here")
            #print(upara[2,:])
            #print(f9)
        else:
            f8 = cross_45_u5[0];
            f9 = cross_45_u5[1];
        
        #objective_func = f1**2 + f2**2 + f3**2 + f4**2 + f5**2 + f6**2 + f7**2 + f8**2 + f9**2;
        #return objective_func
        #print(np.array([f1, f2, f3, f4, f5, f6, f7, f8, f9]));
        return np.array([f1, f2, f3, f4, f5, f6, f7, f8, f9])
    
    #can't use because the cost function is not comfirmed
    def ComputeJac(upara): 
        upara = np.reshape(np.array(upara),(3,3));
        f1_1 = (upara[0,:] - new)/distance(upara[0,:], new);
        f2_1 = (upara[1,:]-fpara[2,:])/distance(upara[1,:], fpara[2,:]);
        f3_1 = new - upara[0,:];
        f5_1 = fpara[2,:] - upara[1,:];
        f7_1 = upara[2,:]/math.sqrt((upara[2,0] - 0)*(upara[2,0] - 0) + (upara[2,1] - 0)*(upara[2,1] - 0) + (upara[2,2] - 0)*(upara[2,2] - 0));
        f8_1 = upara[1,:] - upara[0,:];
    
        Jac = np.array([
            [f1_1[0], f1_1[1], f1_1[2], 0, 0, 0, 0, 0, 0],
            [0, 0, 0, f2_1[0], f2_1[1], f2_1[2], 0, 0, 0],
            [-upara[2,0], -upara[2,1], -upara[2,2], 0, 0, 0, f3_1[0], f3_1[1], f3_1[2]],
            [0, 0, 0, 0, 0, 0, fpara[4,0], fpara[4,1], fpara[4,2]],
            [0, 0, 0, -upara[2,0], -upara[2,1], -upara[2,2], f5_1[0], f5_1[1], f5_1[2]],
            [0, 0, 0, fpara[4,0], fpara[4,1], fpara[4,2], 0, 0, 0],
            [0, 0, 0, 0, 0, 0, f7_1[0], f7_1[1], f7_1[2]],
            [0, 0, 0, 0, 0, 0, 0, -f8_1[2], f8_1[1]],
            [0, 0, 0, 0, 0, 0, f8_1[2], 0, -f8_1[0]]
        ]);
        return Jac
        

    #result = minimize(ConstrainFun, Initial_guess)
    
    #print("Minimum values:", result.x)
    #print("Minimum function value", result.fun); 
    result = root(ConstrainFun, guess)#, jac=ComputeJac)#, options={'maxfev':1000, 'xtol':10e-5}); 
    #result = leastsq(ConstrainFun,guess);
    
    #return result
    #print(result.fun, result.nfev, result.message)
    if result.success:
        solution = result.x
        #print("--------------------------------------")
        return solution, True
    else:
        mes = result.message;
        #print(mes)
        return None, False

# Fixed parameters, the order is 0,1,2,u1,u2
Fixed_para = np.array([
    [-1,0,0], #0 - fixed
    [0,0,0],  #1 - fixed
    [0,0,0],  #2
    [0,0,1],  #u1 - fixed
    [0,0,0]   #u2
], dtype=np.float64);


start_time = time.time()
fullyrotated_num = 0;
temp_count = 0;

########## run start#############
while fullyrotated_num < 1:
    temp_count += 1;
    ######################################### Step 1: Random Picking #########################################################
    # random choose joint2 (x,x,0), can't located at joint1
    Fixed_para[2,0] = random.randint(-3,3);
    Fixed_para[2,1] = random.randint(-3,3);

    while (Fixed_para[2,0] == Fixed_para[1,0]).all():
        luck = np.random.randint(0,1);
        if luck == 0:
            Fixed_para[2,0] = random.randint(-3,3);
        else:
            Fixed_para[2,1] = random.randint(-3,3);

    # random choose joint3 (x,x,0), can't located at joint1 or joint2
    Initial_pos_3 = np.array([np.random.randint(-3,3),np.random.randint(-3,3),0],dtype=np.float64);

    while (Initial_pos_3 == Fixed_para[1,0]).all() or (Initial_pos_3 == Fixed_para[2,0]).all():
        luck = np.random.randint(0,1);
        if luck == 0:
            Initial_pos_3[0]= random.randint(-3,3);
        else:
            Initial_pos_3[1] = random.randint(-3,3);


    #random choose joint4, can't repeat with other joint
    Initial_pos_4 = np.array([random.randint(-3,3),random.randint(-3,3),random.randint(-3,3)],dtype=np.float64);
    while (Initial_pos_4 == Fixed_para[1,0]).all() or (Initial_pos_4 == Fixed_para[2,0]).all() or (Initial_pos_4 == Initial_pos_3).all():
        luck = np.random.randint(0,2);
        if luck == 0:
            Initial_pos_4[0]= random.randint(-3,3);
        elif luck ==1:
            Initial_pos_4[1] = random.randint(-3,3);
        else:
            Initial_pos_4[2] = random.randint(-3,3);
            
    #randomly choose and calculate u2 and u5
    u24 = Initial_pos_4 - Fixed_para[2,:];
    
    #random choose u2, 
    #it DOESN'T has to be perpendiculat to u24
    #Fixed_para[4,:] = RandomChooseVerticalVector(u24);
    Fixed_para[4,:] = np.random.random(size=3);
    Fixed_para[4,:] = Fixed_para[4,:]/np.linalg.norm(Fixed_para[4,:]);
    
    #random choose u5, 
    #it DOESN'T has to be perpendiculat to u24
    #Initial_u5 = RandomChooseVerticalVector(u24);
    Initial_u5 = np.random.random(size=3);
    Initial_u5 = Initial_u5/np.linalg.norm(Initial_u5);
    
    #calculate configuration of the mechanism

    #configuration of the mechanism, the order is l13, l34, l25, 
    #ang_25_u2, ang_25_u5
    #ang_u2_u5, ang_43_u5
    #config_para = np.array([0,0,0,0,0,0,0], dtype=np.float64);
    config_para = ComputeConfig(Fixed_para, Initial_pos_3, Initial_pos_4, Initial_u5);
    
    ######################################## Step 2: Calculate for whole rotation #############################################
    
    Initial_guess = Initial_pos_4, Initial_pos_4, Initial_u5;
    Initial_para = Initial_guess;
    Initial_para, condition = solve_equations(Fixed_para, config_para, Initial_guess, Initial_pos_3);

    
    phi = 0;
    step = 360*16; # this step decides how to devide 2pi
    step_pos_3 = np.zeros((step,3)); # store every position of point_3
    step_para = np.zeros((step,9)); # store every solution
    storage_data = np.zeros((step,27,3)); #store rigidbody data we need
    
    #store the initial data
    step_pos_3[0,:] = Initial_pos_3;
    step_para[0,:] = np.reshape(Initial_para,(1,9));
    #storage_data[0,:,:] = RigidbodyMesh (Initial_para, Initial_pos_3);
   
    u_13 = np.append(0, Initial_pos_3) - np.append(0,Fixed_para[1]);
    
    for i in range(step-1):  
        phi = phi + 360/step;

        #below calculate the next position of 3 by quaternion
        theta = np.pi*phi/360; #thera is half of phi
        rotation_quater = np.array([np.cos(theta), np.sin(theta)*Fixed_para[3,0], np.sin(theta)*Fixed_para[3,1], np.sin(theta)*Fixed_para[3,2]]);#the quaternion of rotation
        new_u_13 = QuaterTimes(QuaterTimes(rotation_quater,u_13),QuaterConj(rotation_quater));
        new_pos_3 = new_u_13[1:4] + Fixed_para[1,:];
        step_pos_3[i+1,:] = new_pos_3;
        
        temp_solution, condition = solve_equations(Fixed_para, config_para, step_para[i,:], new_pos_3);
    
        if condition == True:
            step_para[i+1,:] = np.reshape(temp_solution,(1,9));
        else:
            #print("no full rotation");
            break
    
    if condition == True:
        fullyrotated_num += 1;
        for i in range(step):
            storage_data[i,:,:] = RigidbodyMesh (step_para[i], step_pos_3[i]);
        #print(step_para)
    else:
        continue
########## run end #############

print("--- %s seconds ---" % (time.time() - start_time))
print("Fully rotated machine number: %d" % (fullyrotated_num))
print("Number of attemps: %d" % (temp_count))

In [12]:
# Fixed parameters, the order is 0,1,2,u1,u2
Fixed_para = np.array([
    [-1,0,0], #0 - fixed
    [0,0,0],  #1 - fixed
    [3,0,0],  #2 - fixed
    [0,0,1],  #u1 - fixed
    [-1/math.sqrt(2),0,1/math.sqrt(2)]   #u2 - fixed
], dtype=np.float64);


start_time = time.time()
fullyrotated_num = 0;
temp_count = 0;

########## run start#############
while fullyrotated_num < 1:
    temp_count += 1;
    ######################################### Step 1: Random Picking #########################################################
    # random choose joint3 (x,x,0), can't located at joint1 or joint2
    Initial_pos_3 = np.array([np.random.randint(-3,3),np.random.randint(-3,3),0],dtype=np.float64);

    while (Initial_pos_3 == Fixed_para[1,0]).all() or (Initial_pos_3 == Fixed_para[2,0]).all():
        luck = np.random.randint(0,1);
        if luck == 0:
            Initial_pos_3[0]= random.randint(-3,3);
        else:
            Initial_pos_3[1] = random.randint(-3,3);


    #random choose joint4, can't repeat with other joint
    Initial_pos_4 = np.array([random.randint(-3,3),random.randint(-3,3),random.randint(-3,3)],dtype=np.float64);
    while (Initial_pos_4 == Fixed_para[1,0]).all() or (Initial_pos_4 == Fixed_para[2,0]).all() or (Initial_pos_4 == Initial_pos_3).all():
        luck = np.random.randint(0,2);
        if luck == 0:
            Initial_pos_4[0]= random.randint(-3,3);
        elif luck ==1:
            Initial_pos_4[1] = random.randint(-3,3);
        else:
            Initial_pos_4[2] = random.randint(-3,3);

            
    Initial_u5 = np.array([-1/math.sqrt(2),0,1/math.sqrt(2)]);

    #calculate configuration of the mechanism

    #configuration of the mechanism, the order is l13, l34, l25, 
    #ang_25_u2, ang_25_u5
    #ang_u2_u5, ang_43_u5
    #config_para = np.array([0,0,0,0,0,0,0], dtype=np.float64);
    config_para = ComputeConfig(Fixed_para, Initial_pos_3, Initial_pos_4, Initial_u5);
    
    ######################################## Step 2: Calculate for whole rotation #############################################
    
    Initial_guess = Initial_pos_4, Initial_pos_4, Initial_u5;
    Initial_para = Initial_guess;
    Initial_para, condition = solve_equations(Fixed_para, config_para, Initial_guess, Initial_pos_3);

    
    phi = 0;
    step = 360*16; # this step decides how to devide 2pi
    step_pos_3 = np.zeros((step,3)); # store every position of point_3
    step_para = np.zeros((step,9)); # store every solution
    storage_data = np.zeros((step,27,3)); #store rigidbody data we need
    
    #store the initial data
    step_pos_3[0,:] = Initial_pos_3;
    step_para[0,:] = np.reshape(Initial_para,(1,9));
    #storage_data[0,:,:] = RigidbodyMesh (Initial_para, Initial_pos_3);
   
    u_13 = np.append(0, Initial_pos_3) - np.append(0,Fixed_para[1]);
    
    for i in range(step-1):  
        phi = phi + 360/step;

        #below calculate the next position of 3 by quaternion
        theta = np.pi*phi/360; #thera is half of phi
        rotation_quater = np.array([np.cos(theta), np.sin(theta)*Fixed_para[3,0], np.sin(theta)*Fixed_para[3,1], np.sin(theta)*Fixed_para[3,2]]);#the quaternion of rotation
        new_u_13 = QuaterTimes(QuaterTimes(rotation_quater,u_13),QuaterConj(rotation_quater));
        new_pos_3 = new_u_13[1:4] + Fixed_para[1,:];
        step_pos_3[i+1,:] = new_pos_3;
        
        temp_solution, condition = solve_equations(Fixed_para, config_para, step_para[i,:], new_pos_3);
    
        if condition == True:
            step_para[i+1,:] = np.reshape(temp_solution,(1,9));
        else:
            #print("no full rotation");
            break
    
    if condition == True:
        fullyrotated_num += 1;
        for i in range(step):
            storage_data[i,:,:] = RigidbodyMesh (step_para[i], step_pos_3[i]);
        #print(step_para)
    else:
        continue
########## run end #############

print("--- %s seconds ---" % (time.time() - start_time))
print("Fully rotated machine number: %d" % (fullyrotated_num))
print("Number of attemps: %d" % (temp_count))

C:\Users\84017\.conda\envs\general\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
C:\Users\84017\.conda\envs\general\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


ValueError: cannot reshape array of size 1 into shape (1,9)

In [13]:
Initial_para

In [ ]:
scene = canvas(width=800,height=500,center=vector(0,0,0));


new_pos_3 = step_pos_3[0,:];
upara = step_para[0,:];
#frame
# x_arrow = arrow(pos=vector(0,0,0), axis=vector(10,0,0), shaftwidth=0.2, color=color.red, opacity=0.4); #x
# label(pos=vector(10,0,0), text ='x', xoffset=-2, yoffset=5);
# y_arrow = arrow(pos=vector(0,0,0), axis=vector(0,10,0), shaftwidth=0.2, color=color.green, opacity=0.4); #y
# label(pos=vector(0,10,0), text ='y', xoffset=-2, yoffset=5);
# z_arrow = arrow(pos=vector(0,0,0), axis=vector(0,0,10), shaftwidth=0.2, color=color.blue, opacity=0.4); #z
# label(pos=vector(0,0,10), text ='z', xoffset=-2, yoffset=5);

#joint 1 - fixed joint
joint_f1_front = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(0.5*Fixed_para[3,0],0.5*Fixed_para[3,1],0.5*Fixed_para[3,2]),radius=0.5,color=vector(0.5,0.5,0.5));
joint_f1_back = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(-0.5*Fixed_para[3,0],-0.5*Fixed_para[3,1],-0.5*Fixed_para[3,2]),radius=0.5,color=vector(0.5,0.5,0.5));
#joint 1 - revolute joint
joint_r1_front = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(Fixed_para[3,0],Fixed_para[3,1],Fixed_para[3,2]),radius=0.3,color=color.blue);
joint_r1_back = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(-Fixed_para[3,0],-Fixed_para[3,1],-Fixed_para[3,2]),radius=0.3,color=color.blue);
    
#joint 2 - fixed joint
joint_r2_front = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(0.5*Fixed_para[4,0],0.5*Fixed_para[4,1],0.5*Fixed_para[4,2]),radius=0.5,color=vector(0.5,0.5,0.5));
joint_r2_back = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(-0.5*Fixed_para[4,0],-0.5*Fixed_para[4,1],-0.5*Fixed_para[4,2]),radius=0.5,color=vector(0.5,0.5,0.5));
#joint 2 - revolute joint
joint_r2_front = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(Fixed_para[4,0],Fixed_para[4,1],Fixed_para[4,2]),radius=0.3,color=color.yellow);
joint_r2_back = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(-Fixed_para[4,0],-Fixed_para[4,1],-Fixed_para[4,2]),radius=0.3,color=color.yellow);
    
#joint 3 - spherical joint
joint_s3 = sphere(pos=vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]),radius=0.2);

#joint 4 - connect joint of joint 3 and joint 5
joint_s4 = sphere(pos=vector(upara[0],upara[1],upara[2]),radius=0.15, color=color.red);

#joint 5 - sliding joint
joint_c5_front = cylinder(pos=vector(upara[3],upara[4],upara[5]), axis=vector(0.3*upara[6],0.3*upara[7],0.3*upara[8]),radius=0.3,color=color.yellow);
joint_c5_back = cylinder(pos=vector(upara[3],upara[4],upara[5]), axis=vector(-0.3*upara[6],-0.3*upara[7],-0.3*upara[8]),radius=0.3,color=color.yellow);


#bar 13
vector_13 = -vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2])+vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
bar_13 = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector_13, radius=0.15, color=color.blue);
    
#bar 34
vector_34 = vector(upara[0],upara[1],upara[2])-vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
bar_34 = cylinder(pos=vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]), axis=vector_34, radius=0.15, color=color.red, opacity=0.6);

#bar 45
#vector_45 = 10*(vector(upara[0][3],upara[0][4],upara[0][5])-vector(upara[0][0],upara[0][1],upara[0][2]));
vector_45 = 5*(vector(upara[6],upara[7],upara[8]));
bar_45 = cylinder(pos=vector(upara[0],upara[1],upara[2]), axis=vector_45, radius=0.15, color=color.red, opacity=0.6);
bar_45_back = cylinder(pos=vector(upara[0],upara[1],upara[2]), axis=-vector_45, radius=0.15, color=color.red, opacity=0.6);


#bar 52
vector_52 = -vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2])+vector(upara[3],upara[4],upara[5]);
bar_52 = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector_52, radius=0.15, color=color.yellow);

#Rigid
#for j in range(27):
    #dot = sphere(pos=vector(storage_data[0,j,0],storage_data[0,j,1],storage_data[0,j,2]),radius=0.05,color=color.orange);

for i in range (step):
    rate(1000);
    upara = step_para[i,:];
    new_pos_3 = step_pos_3[i,:];
    joint_s3.pos = vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    joint_s4.pos = vector(upara[0],upara[1],upara[2]);
    joint_c5_front.pos = vector(upara[3],upara[4],upara[5]);
    joint_c5_back.pos = vector(upara[3],upara[4],upara[5]);

    joint_c5_front.axis = vector(0.5*upara[6],0.5*upara[7],0.5*upara[8]);
    joint_c5_back.axis = vector(-0.5*upara[6],-0.5*upara[7],-0.5*upara[8]);

    vector_13 = -vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2])+vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_13.axis = vector_13;

    vector_34 = vector(upara[0],upara[1],upara[2])-vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_34.pos = vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_34.axis = vector_34;

    vector_45 = 5*(vector(upara[6],upara[7],upara[8]));
    bar_45.pos = vector(upara[0],upara[1],upara[2]);
    bar_45.axis = vector_45;
    bar_45_back.pos = vector(upara[0],upara[1],upara[2]);
    bar_45_back.axis = -vector_45;

    vector_52 = -vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2])+vector(upara[3],upara[4],upara[5]);
    bar_52.axis = vector_52;
    
    for j in range (1):
        dot = sphere(pos=vector(storage_data[i,j,0],storage_data[i,j,1],storage_data[i,j,2]),radius=0.05,color=color.orange);

# Non repeat original code - for debug

start_time = time.time()
# Fixed parameters, the order is 0,1,2,u1,u2
Fixed_para = np.array([
    [-1,0,0], #0 - fixed
    [0,0,0],  #1 - fixed
    [0,0,0],  #2
    [0,0,1],  #u1 - fixed
    [0,0,0]   #u2
], dtype=np.float64);

# random choose joint2 (x,x,0), can't located at joint1
Fixed_para[2,0] = random.randint(-3,3);
Fixed_para[2,1] = random.randint(-3,3);

while (Fixed_para[2,0] == Fixed_para[1,0]).all():
    luck = np.random.randint(0,1);
    if luck == 0:
        Fixed_para[2,0] = random.randint(-3,3);
    else:
        Fixed_para[2,1] = random.randint(-3,3);
        
# random choose joint3 (x,x,0), can't located at joint1 or joint2
Initial_pos_3 = np.array([np.random.randint(-3,3),np.random.randint(-3,3),0],dtype=np.float64);

while (Initial_pos_3 == Fixed_para[1,0]).all() or (Initial_pos_3 == Fixed_para[2,0]).all():
    luck = np.random.randint(0,1);
    if luck == 0:
        Initial_pos_3[0]= random.randint(-3,3);
    else:
        Initial_pos_3[1] = random.randint(-3,3);


#random choose joint4, can't repeat with other joint
Initial_pos_4 = np.array([random.randint(-3,3),random.randint(-3,3),random.randint(-3,3)],dtype=np.float64);
while (Initial_pos_4 == Fixed_para[1,0]).all() or (Initial_pos_4 == Fixed_para[2,0]).all() or (Initial_pos_4 == Initial_pos_3).all():
    luck = np.random.randint(0,2);
    if luck == 0:
        Initial_pos_4[0]= random.randint(-3,3);
    elif luck ==1:
        Initial_pos_4[1] = random.randint(-3,3);
    else:
        Initial_pos_4[2] = random.randint(-3,3);

        
#randomly choose and calculate u2 and u5
u24 = Initial_pos_4 - Fixed_para[2,:];
#random choose u2, but it has to be perpendicular to u24
Fixed_para[4,:] = RandomChooseVerticalVector(u24);
Fixed_para[4,:] = Fixed_para[4,:]/np.linalg.norm(Fixed_para[4,:]);
#random choose u5, but it has to be perpendiculat to u24
#Initial_u5 = np.array([0,0,0], dtype=np.float64);
Initial_u5 = RandomChooseVerticalVector(u24);
Initial_u5 = Initial_u5/np.linalg.norm(Initial_u5);


#calculate configuration of the mechanism

#configuration of the mechanism, the order is l13, l34, l25, 
#ang_25_u2, ang_25_u5 - always 90 degree

#ang_u2_u5, ang_34_u5
#config_para = np.array([1,5,6,np.pi*90/180, np.pi*90/180, np.pi*15/180, np.pi*90/180]);
config_para = np.array([0,0,0,np.pi*90/180, np.pi*90/180,0,0]);
config_para[0] = np.linalg.norm(Fixed_para[1,:]-Initial_pos_3); #l13
config_para[1] = np.linalg.norm(Initial_pos_4-Initial_pos_3); #l34
config_para[2] = np.linalg.norm(Fixed_para[2,:]-Initial_pos_4); #l25

#ang_u2_u5
u2_u5 = np.dot(Fixed_para[4,:], Initial_u5)/np.linalg.norm(Fixed_para[4,:]);
u2_u5 = (u2_u5 if u2_u5 > -np.pi else u2_u5+np.pi) if u2_u5 < np.pi else u2_u5-np.pi;#make sure the ang is in ((0,pi)
config_para[5] = np.arccos(u2_u5);

#ang_u34_u5
u34_u5 = np.dot(Initial_pos_3-Initial_pos_4, Initial_u5)/np.linalg.norm(Initial_pos_3-Initial_pos_4);
u34_u5 = (u34_u5 if u34_u5 > -np.pi else u34_u5+np.pi) if u34_u5 < np.pi else u34_u5-np.pi;#make sure the ang is in ((0,pi)
config_para[6] = np.arccos(u34_u5);


Initial_guess = np.array([[1,0,0],[1,0,0],[1,0,0]]);
Initial_guess = Initial_pos_4, Initial_pos_4, Initial_u5
Initial_para = Initial_guess

Initial_para, condition = solve_equations(Fixed_para, config_para, Initial_guess, Initial_pos_3);

config_para_t = config_para;
initial_solution_t = np.reshape(Initial_para,(1,9));

# Step2: 
phi = 0;
step = 360; # this step decides how to devide 2pi
step_para = np.zeros((step,1,9)); # store every position of point_4
step_para[0,:,:] = initial_solution_t;

step_pos_3 = np.zeros((step,3));
step_pos_3[0,:] = Initial_pos_3;

#Initial_pos_3 = Fixed_para[1,:] + np.array([0,config_para_t[0],0]);
u_13 = np.append(0, Initial_pos_3) - np.append(0,Fixed_para[1]);
new_u_13 = u_13;

for i in range(step-1):  
    phi = phi + 360/step;

    #below calculate the next position of 3 by quaternion
    theta = np.pi*phi/360; #thera is half of phi
    rotation_quater = np.array([np.cos(theta), np.sin(theta)*Fixed_para[3,0], np.sin(theta)*Fixed_para[3,1], np.sin(theta)*Fixed_para[3,2]]);#the quaternion of rotation
    new_u_13 = QuaterTimes(QuaterTimes(rotation_quater,u_13),QuaterConj(rotation_quater));
    new_pos_3 = new_u_13[1:4] + Fixed_para[1,:];

    temp_solution, condition = solve_equations(Fixed_para, config_para_t, step_para[i,:,:], new_pos_3);

    if condition == True:
        step_para[i+1,:,:] = np.reshape(temp_solution,(1,9));
        step_pos_3[i+1] = new_pos_3;
    else:
        print("no full rotation");
        break
print(phi)
#print(step_para)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def solve_equations_1(fpara, config, guess, new):
    
    def ConstrainFun (upara):
        upara = np.reshape(np.array(upara),(3,3));
        f1 = distance(new, upara[0,:]) - config[1]; #l_34
        f2 = distance(fpara[2,:], upara[1,:]) - config[2]; #l_25
        f3 = np.dot((new-upara[0,:]),upara[2,:]) - config[1]*np.cos(config[6]); #ang_43_u5
        f4 = np.dot(upara[2,:], fpara[4,:]) - np.cos(config[5]); #ang_u2_u5
        f5 = np.dot(upara[2,:], (fpara[2,:]-upara[1,:])) - config[2]*np.cos(config[4]); #ang_u5_u52
        f6 = np.dot(fpara[4,:], (upara[1,:]-fpara[2,:])) - config[2]*np.cos(config[3]); #ang_u2_u25
        f7 = math.sqrt((upara[2,0] - 0)*(upara[2,0] - 0) + (upara[2,1] - 0)*(upara[2,1] - 0) + (upara[2,2] - 0)*(upara[2,2] - 0)) - 1; #u5 is a unit vector
        #make sure u_45 and u_5 are parallel, f7-f9_2 has 3 valid constraint
        cross_45_u5 = np.cross((upara[1,:]-upara[0,:]),upara[2,:]); #the cross product of u_45 and u_5
        f8 = cross_45_u5[0];
        f9 = cross_45_u5[1];
        if cross_45_u5[2] != 0:
            print("haha")
        #    f9 = cross_45_u5[1];
        #else: #special case that u_35_z = 0, then the f8,f9 can't work
        #    f9 = cross_45_u5[2];
        #objective_func = f1**2 + f2**2 + f3**2 + f4**2 + f5**2 + f6**2 + f7**2 + f8**2 + f9**2;
        
        #return objective_func
        #print(np.array([f1, f2, f3, f4, f5, f6, f7, f8, f9]));
        return np.array([f1, f2, f3, f4, f5, f6, f7, f8, f9])
    
    #can't use because the cost function is not comfirmed
    def ComputeJac(upara): 
        upara = np.reshape(np.array(upara),(3,3));
        f1_1 = (upara[0,:] - new)/distance(upara[0,:], new);
        f2_1 = (upara[1,:]-fpara[2,:])/distance(upara[1,:], fpara[2,:]);
        f3_1 = new - upara[0,:];
        f5_1 = fpara[2,:] - upara[1,:];
        f7_1 = upara[2,:]/math.sqrt((upara[2,0] - 0)*(upara[2,0] - 0) + (upara[2,1] - 0)*(upara[2,1] - 0) + (upara[2,2] - 0)*(upara[2,2] - 0));
        f8_1 = upara[1,:] - upara[0,:];
    
        Jac = np.array([
            [f1_1[0], f1_1[1], f1_1[2], 0, 0, 0, 0, 0, 0],
            [0, 0, 0, f2_1[0], f2_1[1], f2_1[2], 0, 0, 0],
            [-upara[2,0], -upara[2,1], -upara[2,2], 0, 0, 0, f3_1[0], f3_1[1], f3_1[2]],
            [0, 0, 0, 0, 0, 0, fpara[4,0], fpara[4,1], fpara[4,2]],
            [0, 0, 0, -upara[2,0], -upara[2,1], -upara[2,2], f5_1[0], f5_1[1], f5_1[2]],
            [0, 0, 0, fpara[4,0], fpara[4,1], fpara[4,2], 0, 0, 0],
            [0, 0, 0, 0, 0, 0, f7_1[0], f7_1[1], f7_1[2]],
            [0, 0, 0, 0, 0, 0, 0, -f8_1[2], f8_1[1]],
            [0, 0, 0, 0, 0, 0, f8_1[2], 0, -f8_1[0]]
        ]);
        return Jac
        

    #result = minimize(ConstrainFun, Initial_guess)
    
    #print("Minimum values:", result.x)
    #print("Minimum function value", result.fun); 
    result = root(ConstrainFun, guess)#, jac=ComputeJac)#, options={'maxfev':1000, 'xtol':10e-5}); 
    #result = leastsq(ConstrainFun,guess);
    
    #return result
    #print(result.fun, result.nfev, result.message)
    if result.success:
        solution = result.x
        #print("--------------------------------------")
        return solution, True
    else:
        mes = result.message;
        #print(mes)
        return None, False

In [ ]:
# Non repeat original code - for debug

start_time = time.time()
# Fixed parameters, the order is 0,1,2,u1,u2
Fixed_para = np.array([
    [-1,0,0], #0 - fixed
    [0,0,0],  #1 - fixed
    [0,0,0],  #2
    [0,0,1],  #u1 - fixed
    [0,0,0]   #u2
], dtype=np.float64);

# random choose joint2 (x,x,0), can't located at joint1
Fixed_para[2,0] = random.randint(-3,3);
Fixed_para[2,1] = random.randint(-3,3);

while (Fixed_para[2,0] == Fixed_para[1,0]).all():
    luck = np.random.randint(0,1);
    if luck == 0:
        Fixed_para[2,0] = random.randint(-3,3);
    else:
        Fixed_para[2,1] = random.randint(-3,3);
        
# random choose joint3 (x,x,0), can't located at joint1 or joint2
Initial_pos_3 = np.array([np.random.randint(-3,3),np.random.randint(-3,3),0],dtype=np.float64);

while (Initial_pos_3 == Fixed_para[1,0]).all() or (Initial_pos_3 == Fixed_para[2,0]).all():
    luck = np.random.randint(0,1);
    if luck == 0:
        Initial_pos_3[0]= random.randint(-3,3);
    else:
        Initial_pos_3[1] = random.randint(-3,3);


#random choose joint4, can't repeat with other joint
Initial_pos_4 = np.array([random.randint(-3,3),random.randint(-3,3),random.randint(-3,3)],dtype=np.float64);
while (Initial_pos_4 == Fixed_para[1,0]).all() or (Initial_pos_4 == Fixed_para[2,0]).all() or (Initial_pos_4 == Initial_pos_3).all():
    luck = np.random.randint(0,2);
    if luck == 0:
        Initial_pos_4[0]= random.randint(-3,3);
    elif luck ==1:
        Initial_pos_4[1] = random.randint(-3,3);
    else:
        Initial_pos_4[2] = random.randint(-3,3);

        
#randomly choose and calculate u2 and u5
u24 = Initial_pos_4 - Fixed_para[2,:];
#random choose u2, but it has to be perpendicular to u24
Fixed_para[4,:] = RandomChooseVerticalVector(u24);
Fixed_para[4,:] = Fixed_para[4,:]/np.linalg.norm(Fixed_para[4,:]);
#random choose u5, but it has to be perpendiculat to u24
#Initial_u5 = np.array([0,0,0], dtype=np.float64);
Initial_u5 = RandomChooseVerticalVector(u24);
Initial_u5 = Initial_u5/np.linalg.norm(Initial_u5);


#calculate configuration of the mechanism

#configuration of the mechanism, the order is l13, l34, l25, 
#ang_25_u2, ang_25_u5 - always 90 degree

#ang_u2_u5, ang_34_u5
#config_para = np.array([1,5,6,np.pi*90/180, np.pi*90/180, np.pi*15/180, np.pi*90/180]);
config_para = np.array([0,0,0,np.pi*90/180, np.pi*90/180,0,0]);
config_para[0] = np.linalg.norm(Fixed_para[1,:]-Initial_pos_3); #l13
config_para[1] = np.linalg.norm(Initial_pos_4-Initial_pos_3); #l34
config_para[2] = np.linalg.norm(Fixed_para[2,:]-Initial_pos_4); #l25

#ang_u2_u5
u2_u5 = np.dot(Fixed_para[4,:], Initial_u5)/np.linalg.norm(Fixed_para[4,:]);
u2_u5 = (u2_u5 if u2_u5 > -np.pi else u2_u5+np.pi) if u2_u5 < np.pi else u2_u5-np.pi;#make sure the ang is in ((0,pi)
config_para[5] = np.arccos(u2_u5);

#ang_u34_u5
u34_u5 = np.dot(Initial_pos_3-Initial_pos_4, Initial_u5)/np.linalg.norm(Initial_pos_3-Initial_pos_4);
u34_u5 = (u34_u5 if u34_u5 > -np.pi else u34_u5+np.pi) if u34_u5 < np.pi else u34_u5-np.pi;#make sure the ang is in ((0,pi)
config_para[6] = np.arccos(u34_u5);


In [ ]:
Initial_guess = np.array([[1,0,0],[1,0,0],[1,0,0]]);
Initial_guess = Initial_pos_4, Initial_pos_4, Initial_u5
Initial_para = Initial_guess

Initial_para, condition = solve_equations(Fixed_para, config_para, Initial_guess, Initial_pos_3);

config_para_t = config_para;
initial_solution_t = np.reshape(Initial_para,(1,9));

# Step2: 
phi = 0;
step = 360; # this step decides how to devide 2pi
step_para = np.zeros((step,1,9)); # store every position of point_4
step_para[0,:,:] = initial_solution_t;

step_pos_3 = np.zeros((step,3));
step_pos_3[0,:] = Initial_pos_3;

#Initial_pos_3 = Fixed_para[1,:] + np.array([0,config_para_t[0],0]);
u_13 = np.append(0, Initial_pos_3) - np.append(0,Fixed_para[1]);
new_u_13 = u_13;

for i in range(step-1):  
    phi = phi + 360/step;

    #below calculate the next position of 3 by quaternion
    theta = np.pi*phi/360; #thera is half of phi
    rotation_quater = np.array([np.cos(theta), np.sin(theta)*Fixed_para[3,0], np.sin(theta)*Fixed_para[3,1], np.sin(theta)*Fixed_para[3,2]]);#the quaternion of rotation
    new_u_13 = QuaterTimes(QuaterTimes(rotation_quater,u_13),QuaterConj(rotation_quater));
    new_pos_3 = new_u_13[1:4] + Fixed_para[1,:];

    temp_solution, condition = solve_equations(Fixed_para, config_para_t, step_para[i,:,:], new_pos_3);

    if condition == True:
        step_para[i+1,:,:] = np.reshape(temp_solution,(1,9));
        step_pos_3[i+1] = new_pos_3;
    else:
        print("no full rotation");
        break
print(phi)
#print(step_para)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
scene = canvas(width=800,height=500,center=vector(0,0,0));

new_pos_3 = step_pos_3[0,:];
upara = step_para[0,:];
#frame
x_arrow = arrow(pos=vector(0,0,0), axis=vector(10,0,0), shaftwidth=0.2, color=color.red, opacity=0.4); #x
label(pos=vector(10,0,0), text ='x', xoffset=-2, yoffset=5);
y_arrow = arrow(pos=vector(0,0,0), axis=vector(0,10,0), shaftwidth=0.2, color=color.green, opacity=0.4); #y
label(pos=vector(0,10,0), text ='y', xoffset=-2, yoffset=5);
z_arrow = arrow(pos=vector(0,0,0), axis=vector(0,0,10), shaftwidth=0.2, color=color.blue, opacity=0.4); #z
label(pos=vector(0,0,10), text ='z', xoffset=-2, yoffset=5);

#joint 1 - fixed joint
joint_f1_front = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(0.5*Fixed_para[3,0],0.5*Fixed_para[3,1],0.5*Fixed_para[3,2]),radius=0.5,color=vector(0.5,0.5,0.5));
joint_f1_back = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(-0.5*Fixed_para[3,0],-0.5*Fixed_para[3,1],-0.5*Fixed_para[3,2]),radius=0.5,color=vector(0.5,0.5,0.5));
#joint 1 - revolute joint
joint_r1_front = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(Fixed_para[3,0],Fixed_para[3,1],Fixed_para[3,2]),radius=0.3,color=color.blue);
joint_r1_back = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector(-Fixed_para[3,0],-Fixed_para[3,1],-Fixed_para[3,2]),radius=0.3,color=color.blue);
    
#joint 2 - fixed joint
joint_r2_front = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(0.5*Fixed_para[4,0],0.5*Fixed_para[4,1],0.5*Fixed_para[4,2]),radius=0.5,color=vector(0.5,0.5,0.5));
joint_r2_back = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(-0.5*Fixed_para[4,0],-0.5*Fixed_para[4,1],-0.5*Fixed_para[4,2]),radius=0.5,color=vector(0.5,0.5,0.5));
#joint 2 - revolute joint
joint_r2_front = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(Fixed_para[4,0],Fixed_para[4,1],Fixed_para[4,2]),radius=0.3,color=color.yellow);
joint_r2_back = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector(-Fixed_para[4,0],-Fixed_para[4,1],-Fixed_para[4,2]),radius=0.3,color=color.yellow);
    
#joint 3 - spherical joint
joint_s3 = sphere(pos=vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]),radius=0.2);

#joint 4 - connect joint of joint 3 and joint 5
joint_s4 = sphere(pos=vector(upara[0,0],upara[0,1],upara[0,2]),radius=0.15, color=color.red);

#joint 5 - sliding joint
joint_c5_front = cylinder(pos=vector(upara[0,3],upara[0,4],upara[0,5]), axis=vector(0.3*upara[0,6],0.3*upara[0,7],0.3*upara[0,8]),radius=0.3,color=color.yellow);
joint_c5_back = cylinder(pos=vector(upara[0,3],upara[0,4],upara[0,5]), axis=vector(-0.3*upara[0,6],-0.3*upara[0,7],-0.3*upara[0,8]),radius=0.3,color=color.yellow);


#bar 13
vector_13 = -vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2])+vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
bar_13 = cylinder(pos=vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2]), axis=vector_13, radius=0.15, color=color.blue);
    
#bar 34
vector_34 = vector(upara[0,0],upara[0,1],upara[0,2])-vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
bar_34 = cylinder(pos=vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]), axis=vector_34, radius=0.15, color=color.red, opacity=0.6);

#bar 45
#vector_45 = 10*(vector(upara[0][3],upara[0][4],upara[0][5])-vector(upara[0][0],upara[0][1],upara[0][2]));
vector_45 = 5*(vector(upara[0,6],upara[0,7],upara[0,8]));
bar_45 = cylinder(pos=vector(upara[0,0],upara[0,1],upara[0,2]), axis=vector_45, radius=0.15, color=color.red, opacity=0.6);
bar_45_back = cylinder(pos=vector(upara[0,0],upara[0,1],upara[0,2]), axis=-vector_45, radius=0.15, color=color.red, opacity=0.6);

#bar 52
vector_52 = -vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2])+vector(upara[0,3],upara[0,4],upara[0,5]);
bar_52 = cylinder(pos=vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2]), axis=vector_52, radius=0.15, color=color.yellow);


for i in range (1,step):
    rate(40);
    upara = step_para[i,:];
    new_pos_3 = step_pos_3[i,:];
    joint_s3.pos = vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    joint_s4.pos = vector(upara[0,0],upara[0,1],upara[0,2]);
    joint_c5_front.pos = vector(upara[0,3],upara[0,4],upara[0,5]);
    joint_c5_back.pos = vector(upara[0,3],upara[0,4],upara[0,5]);

    joint_c5_front.axis = vector(0.5*upara[0,6],0.5*upara[0,7],0.5*upara[0,8]);
    joint_c5_back.axis = vector(-0.5*upara[0,6],-0.5*upara[0,7],-0.5*upara[0,8]);

    vector_13 = -vector(Fixed_para[1,0],Fixed_para[1,1],Fixed_para[1,2])+vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_13.axis = vector_13;

    vector_34 = vector(upara[0,0],upara[0,1],upara[0,2])-vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_34.pos = vector(new_pos_3[0],new_pos_3[1],new_pos_3[2]);
    bar_34.axis = vector_34;

    vector_45 = 5*(vector(upara[0,6],upara[0,7],upara[0,8]));
    bar_45.pos = vector(upara[0,0],upara[0,1],upara[0,2]);
    bar_45.axis = vector_45;
    bar_45_back.pos = vector(upara[0,0],upara[0,1],upara[0,2]);
    bar_45_back.axis = -vector_45;

    vector_52 = -vector(Fixed_para[2,0],Fixed_para[2,1],Fixed_para[2,2])+vector(upara[0,3],upara[0,4],upara[0,5]);
    bar_52.axis = vector_52;
    
    #for j in range (2):
        #dot = sphere(pos=vector(storage_data[i,j,0],storage_data[i,j,1],storage_data[i,j,2]),radius=0.05,color=color.orange);

In [ ]:
del step_para